In [ ]:
import struct

import numpy as np
import fitsio
import matplotlib.pyplot as plt

from astropy.cosmology import FlatLambdaCDM

# import qsotools.fiducial as qfid

# plt.style.use("/Users/nk452/style_prof.mplstyle")

In [ ]:
def turner24_mf(z):
    return np.exp(-2.46e-3 * (1 + z)**3.62)

In [ ]:
class SherwoodGrid(object):
    """
    Parameters:
    fname (str): Filename to open
    a (int, default: 1): Axis to pick
    
    Additional notes on data structure
    Header:
    npix (long, 4 bytes)
    nlos (long, 4 bytes)
    ztime (float, 4 bytes)
    omegaM (float, 4 bytes)
    omegaL (float, 4 bytes)
    omegab (float, 4 bytes)
    hubble, h (float, 4 bytes)
    boxsize, h/ckpc (float, 4 bytes)
    
    Data:
    axis[nlos] (long, 4 * nlos)
    coord1[nlos] (float, 4 * nlos)
    coord2[nlos] (float, 4 * nlos)
    pixels[npix] (float, 4 * npix)
    tau[npix * nlos] (float, 4 * nlos * npix)
    """

    HDR = "iiffffff"
    HDR_SIZE = struct.calcsize(HDR)

    def __init__(self, fname):
        with open(fname, "rb") as f:
            (self.npix, self.nlos, self.ztime,
             self.omegaM, self.omegaL, self.omegab,
             self.h, self.L
            ) = struct.unpack(SherwoodGrid.HDR, f.read(SherwoodGrid.HDR_SIZE))
            
            self.axis = np.fromfile(
                f, dtype='i4', count=self.nlos)
            self.coord1 = np.fromfile(
                f, dtype='f4', count=self.nlos) / 1e3
            self.coord2 = np.fromfile(
                f, dtype='f4', count=self.nlos) / 1e3
            self.pixels = np.fromfile(
                f, dtype='f4', count=self.npix) / 1e3
            self.tau = np.fromfile(
                f, dtype='f4', count=self.npix * self.nlos)
        
        self.L /= 1e3  # in Mpc / h
        self.coord1 = self.coord1
        self.coord2 = self.coord2
        self.tau = self.tau.reshape(self.nlos, self.npix)
        self.flux = np.exp(-self.tau)
        self.meanflux = np.mean(self.flux)
        self.deltas = self.flux / self.meanflux - 1
        self.cosmo = FlatLambdaCDM(
            H0=100 * self.h, Om0=self.omegaM, Ob0=self.omegab)
        self.cMpch2kms = 100.0 * self.cosmo.efunc(self.ztime) / (1 + self.ztime)

    def estimateP1D(self):
        dlos = self.pixels[1] - self.pixels[0]
        klos = 2 * np.pi * np.fft.rfftfreq(self.npix, d=dlos)
        deltas_k = np.fft.rfft(self.deltas, axis=1) * dlos
        p1d = np.abs(deltas_k)**2 / self.L
        return klos, p1d.mean(axis=0)

In [ ]:
fts_sw = SherwoodGrid("tauH1_lya_z2.8.dat")

In [ ]:
plt.plot(fts_sw.pixels, np.exp(-fts_sw.tau[0]))
plt.plot(fts_sw.pixels, np.exp(-fts_sw.tau[10]))
plt.show()